# Organelle Morpholgy analysis

In the following we will demonstrate the `organelle_morphology` package.

In [ ]:
import organelle_morphology

In [ ]:
project_path = "../data/cebra_em_example/seg_er_5nm_mito_10nm/"

In [ ]:
p = organelle_morphology.Project(project_path)

In [ ]:
p.available_sources()

In [ ]:
p.add_source("mito-it00_b0_6_stitched", organelle="mito")
p.compression_level = 2

In [ ]:
geo = p.geometric_properties
geo.head(5)

In [ ]:
# filter only water tight meshes
ids = geo[geo["water_tight"] == True].index.to_list()

In [ ]:
p.generate_skeletons(ids=ids)

In [ ]:
p.show(ids=ids, show_skeleton=True, height=1000)

In [ ]:
org = p.organelles("*0346")[0]
org

In [ ]:
import skeletor as sk

In [ ]:
fixed_mesh = sk.pre.fix_mesh(org.mesh)
cont = sk.pre.contract(fixed_mesh, epsilon=0.05)
skel = sk.skeletonize.by_vertex_clusters(cont, sampling_dist=2)
skel.mesh = fixed_mesh
sk.post.clean_up(skel, inplace=True, theta=0.8, remove_short_branches=100)
skel

In [ ]:
sk.post.clean_up(skel, inplace=True, theta=0.4)

In [ ]:
import plotly.graph_objects as go


import plotly.graph_objects as go

# Assuming nodes is a list of tuples (x, y) and edges is a list of tuples (node1, node2)
nodes = skel.vertices
edges = skel.edges

line_width = 10

# Create a 3D line plot for the edges
x_values = []
y_values = []
z_values = []

for edge in edges:
    x_values.extend(
        [nodes[edge[0]][0], nodes[edge[1]][0], None]
    )  # add None to separate lines
    y_values.extend(
        [nodes[edge[0]][1], nodes[edge[1]][1], None]
    )  # add None to separate lines
    z_values.extend(
        [nodes[edge[0]][2], nodes[edge[1]][2], None]
    )  # add None to separate lines

edge_trace = go.Scatter3d(
    x=x_values,
    y=y_values,
    z=z_values,
    mode="lines",
    line=dict(width=line_width),  # Set line width
    name="Edge",  # Set label
)

In [ ]:
p.show(ids="*0346", show_morphology=False, show_skeleton=True, extra_trace=edge_trace)

In [ ]:
skel.edges
import numpy as np

orig_vertices = skel.vertices

# the sample points are points along the skeleton arms
# and the reference points are the vertices of the skeleton from which these samples have been generated.
# we need these to later calculate the normal vector for the plane which will intersect our mesh
sampled_path = []
reference_point = []

distance_between_points = 0.1

for edge in skel.edges:
    edge_len = np.linalg.norm(
        np.array(skel.vertices[edge[0]]) - np.array(skel.vertices[edge[1]])
    )
    if edge_len > distance_between_points:
        p1 = np.array(skel.vertices[edge[0]])
        p2 = np.array(skel.vertices[edge[1]])

        # find number of points to add bewteen the two vertices
        n_points = np.ceil(edge_len / distance_between_points).astype(int)
        factors = np.linspace(0, 1, n_points)

        # Compute the interpolated points
        interpolated_points = (1 - factors[:, np.newaxis]) * p1 + factors[
            :, np.newaxis
        ] * p2
        sampled_path.extend(interpolated_points)

sampled_path = np.asarray(sampled_path)

In [ ]:
orig_vertices

In [ ]:
import plotly.graph_objects as go

scatter1 = go.Scatter3d(
    x=orig_vertices[:, 0], y=orig_vertices[:, 1], z=orig_vertices[:, 2], mode="markers"
)
scatter2 = go.Scatter3d(
    x=sampled_path[:, 0], y=sampled_path[:, 1], z=sampled_path[:, 2], mode="markers"
)

In [ ]:
fig = go.Figure()
fig.add_trace(edge_trace)
fig.add_trace(scatter1)
fig.add_trace(scatter2)